In [3]:
!pip install cohere gtts moviepy python-telegram-bot==13.14


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.4/497.4 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.6/514.6 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.9 MB/s eta 0:00:00
  Created wheel for tornado: filename=tornado-6.1-cp310-cp310-linux_x86_64.whl size=421979 sha256=bbe8bf6addfdf9c43b279c6622b6a749b80821ed5dc3154bc2dca1e20c7ff1bb
  Stored in directory: /root/.cache/pip/wheels/80/32/8d/21cf0fa6ee4e083f6530e5b83dfdfa9489a3890d320803f4c7
Successfully built tornado
  Attempting uninstall: tornado
    Found existing installation: tornado 6.3.3
    Uninstalling tornado-6.3.3:
      Successfully uninstalled tornado-6.3.3
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.4.0
    Uninstalling cachetools-5.4.0:
      Successfully uninstalled cachetools-5.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are 

In [40]:
import cohere
from gtts import gTTS
from moviepy.editor import ImageClip, AudioFileClip
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from telegram import InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import CallbackContext, CallbackQueryHandler
from telegram.update import Update

In [41]:
co = cohere.Client('msBTuCyK09FRruQx95UAJ3BYKAJEESoshz3f4o8i')

In [42]:
# Define the Cohere response generation function
def generate_response(question):
    response = co.generate(
        model='command-xlarge-nightly',
        prompt=question,
        max_tokens=150
    )
    return response.generations[0].text.strip()

In [43]:

# Define the Text-to-Speech function
def text_to_speech(text, output_file):
    tts = gTTS(text=text, lang='en')
    tts.save(output_file)

In [46]:
# Define the Video Generation function
def generate_video(audio_file, output_file):
    video_clip = ImageClip("sahithi_img.jpg").set_duration(AudioFileClip(audio_file).duration)
    audio_clip = AudioFileClip(audio_file)
    video_clip = video_clip.set_audio(audio_clip)
    video_clip.write_videofile(output_file, codec='libx264', fps=24)


In [47]:

# Main function to handle Telegram messages
def handle_question(update: Update, context: CallbackContext):
    question = update.message.text
    response = generate_response(question)

    audio_file = "response.mp3"
    video_file = "response_video.mp4"

    text_to_speech(response, audio_file)
    generate_video(audio_file, video_file)

    update.message.reply_text("AI Sahithi response: " + response)
    context.bot.send_video(chat_id=update.message.chat_id, video=open(video_file, 'rb'))

In [48]:

def start(update: Update, context: CallbackContext):
    update.message.reply_text("Hello! I am AI Sahithi. How can I assist you today?")

In [49]:

def help_command(update: Update, context: CallbackContext):
    update.message.reply_text("You can ask me any question!")

In [50]:

def unknown(update: Update, context: CallbackContext):
    update.message.reply_text("Sorry, I didn't understand that command.")

In [51]:
# Setup the Telegram bot
updater = Updater("6704753283:AAGQZ0SnvRjpPfXoDqVLtvCFNUI70SKDLwU", use_context=True)
dispatcher = updater.dispatcher

In [52]:
# Add handlers for different commands and messages
dispatcher.add_handler(CommandHandler('start', start))
dispatcher.add_handler(CommandHandler('help', help_command))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_question))
dispatcher.add_handler(MessageHandler(Filters.command, unknown))


In [54]:
updater.start_polling()

In [39]:
updater.stop()